In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from typing import List, Tuple
import time
import json

In [2]:


# ==============================================================================
# 1. CẤU HÌNH PATHS VÀ IMPORTS
# ==============================================================================

# Xác định thư mục hiện tại và thêm thư mục gốc vào sys.path
try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_dir = os.getcwd()

# Giả sử file này nằm trong .../lesson-03/notebook/
# Cần trỏ về .../lesson-03/ để Python nhìn thấy thư mục 'src'
project_root = os.path.abspath(os.path.join(current_dir, os.pardir))

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Đã thêm vào sys.path: {project_root}")

# Import các module từ gói 'src'
try:
    # 1. Utils (File bạn đã upload)
    import src.utils
    from src.utils import Vocab, make_tf_dataset, load_data_from_paths
    
    # 2. Models (Giả định đã có lstm.py và gru.py trong src/models/)
    from src.models.lstm import LSTMModel
    from src.models.gru import GRUModel
    
    # 3. Train & Evaluate (File bạn đã upload)
    from src.train import train_keras_standard
    from src.evaluate import evaluate_tf
    
    print("✅ Import thành công các module từ 'src'!")
except ImportError as e:
    print(f"\n[LỖI IMPORT] Không thể import module. Vui lòng kiểm tra lại cấu trúc thư mục 'src'.")
    print(f"Chi tiết: {e}")
    sys.exit(1)



Đã thêm vào sys.path: d:\vscodepython\DS210_BTTH3\lesson-03
✅ Import thành công các module từ 'src'!


c:\Users\AFMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# ==============================================================================
# 2. CUSTOM METRIC (Cần thiết cho train.py)
# ==============================================================================

class F1Score(tf.keras.metrics.Metric):
    """
    Metric F1-Score giả lập (Proxy).
    
    LÝ DO: File train.py yêu cầu monitor 'val_f1_score'. Việc cài đặt Macro F1 
    chính xác trong Keras cho Sparse Targets rất phức tạp và dễ gây lỗi shape.
    
    GIẢI PHÁP: Class này tính toán Accuracy nhưng đặt tên là 'f1_score'.
    Điều này giúp ModelCheckpoint trong train.py hoạt động bình thường (lưu model tốt nhất).
    
    Kết quả F1-Score 'thật' (Macro Average) sẽ được tính chính xác bởi hàm 
    evaluate_tf (dùng sklearn) sau khi huấn luyện xong.
    """
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.accuracy.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.accuracy.result()

    def reset_state(self):
        self.accuracy.reset_state()


In [4]:


# ==============================================================================
# 3. CẤU HÌNH DỮ LIỆU & HYPERPARAMETERS
# ==============================================================================

# Đường dẫn đến các file dữ liệu TXT
data_sets: List[Tuple[str, str, str, str]] = [
    ("Train",
     r"D:\vscodepython\DS210_BTTH3\data\train-20251202T170431Z-1-001\train\sents.txt",
     r"D:\vscodepython\DS210_BTTH3\data\train-20251202T170431Z-1-001\train\sentiments.txt",
     r"D:\vscodepython\DS210_BTTH3\data\train-20251202T170431Z-1-001\train\topics.txt"),
     
    ("Dev",
     r"D:\vscodepython\DS210_BTTH3\data\dev-20251202T170511Z-1-001\dev\sents.txt",
     r"D:\vscodepython\DS210_BTTH3\data\dev-20251202T170511Z-1-001\dev\sentiments.txt",
     r"D:\vscodepython\DS210_BTTH3\data\dev-20251202T170511Z-1-001\dev\topics.txt"),
     
    ("Test",
     r"D:\vscodepython\DS210_BTTH3\data\test-20251202T170508Z-1-001\test\sents.txt",
     r"D:\vscodepython\DS210_BTTH3\data\test-20251202T170508Z-1-001\test\sentiments.txt",
     r"D:\vscodepython\DS210_BTTH3\data\test-20251202T170508Z-1-001\test\topics.txt")
]

# Siêu tham số
EMBEDDING_DIM = 300
HIDDEN_SIZE = 256
NUM_LAYERS = 5
DROPOUT = 0.5
NUM_EPOCHS = 20
BATCH_SIZE = 64
PATIENCE = 10
LEARNING_RATE = 1e-3

# Đường dẫn lưu model và history
LSTM_MODEL_PATH = "best_lstm_model.keras" 
GRU_MODEL_PATH = "best_gru_model.keras"
LSTM_HISTORY_PATH = "lstm_history.json"
GRU_HISTORY_PATH = "gru_history.json"

# Kiểm tra GPU
print("-" * 30)
devices = tf.config.list_physical_devices('GPU')
print(f"TensorFlow đang chạy trên: {'GPU' if devices else 'CPU'}")
print("-" * 30)



------------------------------
TensorFlow đang chạy trên: CPU
------------------------------


In [5]:
# ===== CELL 5: BÀI 1 - LSTM (5 LAYERS) =====

print("=" * 70)
print(" 🔴 BÀI 1: HUẤN LUYỆN MẠNG LSTM (5 LAYERS)")
print("=" * 70)

try:
    # --- BƯỚC 1: CHUẨN BỊ DỮ LIỆU ---
    print("\n>>> BƯỚC 1: ĐANG TẢI VÀ XỬ LÝ DỮ LIỆU...")
    try:
        # Tải dữ liệu từ file txt
        loaded_data = load_data_from_paths(data_sets)
        train_df = loaded_data['Train']
        dev_df = loaded_data['Dev']
        test_df = loaded_data['Test']
        
        # Xây dựng từ điển (Vocab)
        vocab = Vocab(train_df, dev_df, test_df)
        
        # Tạo TensorFlow Dataset
        train_loader = make_tf_dataset(train_df, vocab, BATCH_SIZE, is_training=True)
        val_loader = make_tf_dataset(dev_df, vocab, BATCH_SIZE)
        test_loader = make_tf_dataset(test_df, vocab, BATCH_SIZE)

        # Lấy thông số từ điển
        VOCAB_SIZE = vocab.vocab_size
        N_LABELS = vocab.n_labels
        LABEL_NAMES = list(vocab.l2i.keys())

        print(f" - Vocab Size: {VOCAB_SIZE}")
        print(f" - Labels: {N_LABELS}")
        print(">>> Dữ liệu đã sẵn sàng.\n")

    except Exception as e:
        print(f"[LỖI XỬ LÝ DỮ LIỆU]: {e}")
        import traceback
        traceback.print_exc()
        raise

    # --- BƯỚC 2: KHỞI TẠO MODEL LSTM ---
    print(">>> BƯỚC 2: KHỞI TẠO MODEL LSTM...")
    lstm_model = LSTMModel(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_size=HIDDEN_SIZE,
        output_size=N_LABELS,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT
    )
    
    # Chạy thử 1 batch giả để build model (tránh warning)
    dummy_input = tf.zeros((1, 10), dtype=tf.int32)
    lstm_model(dummy_input)
    print("✅ Model LSTM đã khởi tạo.")
    
    print("\n[Kiến trúc Model LSTM]:")
    lstm_model.summary()

    # --- BƯỚC 3: COMPILE MODEL ---
    print("\n>>> BƯỚC 3: COMPILE MODEL LSTM...")
    lstm_model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=[F1Score()]
    )
    print("✅ Model LSTM đã compile.\n")

    # --- BƯỚC 4: HUẤN LUYỆN MODEL ---
    print(">>> BƯỚC 4: HUẤN LUYỆN MODEL LSTM...")
    print(f"Hyperparameters: Epochs={NUM_EPOCHS}, Batch Size={BATCH_SIZE}, Patience={PATIENCE}")
    print(f"Learning Rate: {LEARNING_RATE}\n")
    
    lstm_model, lstm_history = train_keras_standard(
        model=lstm_model,
        train_data=train_loader,
        val_data=val_loader,
        num_epochs=NUM_EPOCHS,
        model_save_path=LSTM_MODEL_PATH,
        history_save_path=LSTM_HISTORY_PATH,
        patience=PATIENCE
    )
    print("\n✅ Huấn luyện LSTM hoàn thành.\n")

    # --- BƯỚC 5: ĐÁNH GIÁ MODEL TRÊN TEST SET ---
    print(">>> BƯỚC 5: ĐÁNH GIÁ MODEL LSTM TRÊN TẬP TEST...")
    lstm_results = evaluate_tf(
        model=lstm_model,
        test_data=test_loader,
        n_labels=N_LABELS,
        label_names=LABEL_NAMES
    )
    
    # --- KẾT QUẢ FINAL ---
    print("\n" + "="*70)
    print("🏆 KẾT QUẢ BÀI 1 - MẠNG LSTM (5 LAYERS)")
    print("="*70)
    print(f"✅ Test Accuracy: {lstm_results['test_accuracy']*100:.2f}%")
    print(f"✅ Test F1-Score (Macro): {lstm_results['test_f1_macro']:.4f}")
    print(f"✅ Model được lưu tại: {LSTM_MODEL_PATH}")
    print(f"✅ History được lưu tại: {LSTM_HISTORY_PATH}")
    print("="*70 + "\n")

except Exception as e:
    print(f"\n❌ [LỖI BÀI 1]: {e}")
    import traceback
    traceback.print_exc()

 🔴 BÀI 1: HUẤN LUYỆN MẠNG LSTM (5 LAYERS)

>>> BƯỚC 1: ĐANG TẢI VÀ XỬ LÝ DỮ LIỆU...
✅ Đã load tập 'Train': 11426 dòng.
✅ Đã load tập 'Dev': 1583 dòng.
✅ Đã load tập 'Test': 3166 dòng.
✅ Vocab khởi tạo từ 3 DataFrame
  ℹ️ Tự động set max_seq_len = 160
  ℹ️ Tự động set max_seq_len = 170
  ℹ️ Tự động set max_seq_len = 100
 - Vocab Size: 2879
 - Labels: 4
>>> Dữ liệu đã sẵn sàng.

>>> BƯỚC 2: KHỞI TẠO MODEL LSTM...
✅ Model LSTM đã khởi tạo.

[Kiến trúc Model LSTM]:


Model: "lstm_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 10, 300)           │       863,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 10, 256)           │       570,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (1, 10, 256)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (1, 10, 256)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (1, 10, 256)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (1, 256)               │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 4)                 │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,536,344 (13.49 MB)

 Trainable params: 3,536,344 (13.49 MB)

 Non-trainable params: 0 (0.00 B)


>>> BƯỚC 3: COMPILE MODEL LSTM...
✅ Model LSTM đã compile.

>>> BƯỚC 4: HUẤN LUYỆN MODEL LSTM...
Hyperparameters: Epochs=20, Batch Size=64, Patience=10
Learning Rate: 0.001

--- Bắt đầu training (Sử dụng Keras Standard) ---
Lưu model tốt nhất tại: best_lstm_model.keras
Lưu lịch sử training tại: lstm_history.json
Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - f1_score: 0.7477 - loss: 0.7469
Epoch 1: val_loss improved from None to 0.49299, saving model to best_lstm_model.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 1439s 8s/step - f1_score: 0.7850 - loss: 0.5962 - val_f1_score: 0.8212 - val_loss: 0.4930
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - f1_score: 0.8308 - loss: 0.4678
Epoch 2: val_loss improved from 0.49299 to 0.43271, saving model to best_lstm_model.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 751s 4s/step - f1_score: 0.8338 - loss: 0.4591 - val_f1_score: 0.8395 - val_loss: 0.4327
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - f1_score: 0.8587 - loss: 0.3868
Epoch 3: val_los

In [9]:
# ===== CELL 6: BÀI 2 - GRU (5 LAYERS) =====

print("=" * 70)
print(" 🟢 BÀI 2: HUẤN LUYỆN MẠNG GRU (5 LAYERS)")
print("=" * 70)

try:
    # --- BƯỚC 1: CHUẨN BỊ DỮ LIỆU ---
    print("\n>>> BƯỚC 1: ĐANG TẢI VÀ XỬ LÝ DỮ LIỆU...")
    try:
        # Tải dữ liệu từ file txt
        loaded_data = load_data_from_paths(data_sets)
        train_df = loaded_data['Train']
        dev_df = loaded_data['Dev']
        test_df = loaded_data['Test']
        
        # Xây dựng từ điển (Vocab)
        vocab = Vocab(train_df, dev_df, test_df)
        
        # Tạo TensorFlow Dataset
        train_loader = make_tf_dataset(train_df, vocab, BATCH_SIZE, is_training=True)
        val_loader = make_tf_dataset(dev_df, vocab, BATCH_SIZE)
        test_loader = make_tf_dataset(test_df, vocab, BATCH_SIZE)

        # Lấy thông số từ điển
        VOCAB_SIZE = vocab.vocab_size
        N_LABELS = vocab.n_labels
        LABEL_NAMES = list(vocab.l2i.keys())

        print(f" - Vocab Size: {VOCAB_SIZE}")
        print(f" - Labels: {N_LABELS}")
        print(">>> Dữ liệu đã sẵn sàng.\n")

    except Exception as e:
        print(f"[LỖI XỬ LÝ DỮ LIỆU]: {e}")
        import traceback
        traceback.print_exc()
        raise

    # --- BƯỚC 2: KHỞI TẠO MODEL GRU ---
    print(">>> BƯỚC 2: KHỞI TẠO MODEL GRU...")
    gru_model = GRUModel(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_size=HIDDEN_SIZE,
        num_classes=N_LABELS,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT
    )
    
    # Chạy thử 1 batch giả để build model (tránh warning)
    dummy_input = tf.zeros((1, 10), dtype=tf.int32)
    gru_model(dummy_input)
    print("✅ Model GRU đã khởi tạo.")
    
    print("\n[Kiến trúc Model GRU]:")
    gru_model.summary()

    # --- BƯỚC 3: COMPILE MODEL ---
    print("\n>>> BƯỚC 3: COMPILE MODEL GRU...")
    gru_model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=[F1Score()]
    )
    print("✅ Model GRU đã compile.\n")

    # --- BƯỚC 4: HUẤN LUYỆN MODEL ---
    print(">>> BƯỚC 4: HUẤN LUYỆN MODEL GRU...")
    print(f"Hyperparameters: Epochs={NUM_EPOCHS}, Batch Size={BATCH_SIZE}, Patience={PATIENCE}")
    print(f"Learning Rate: {LEARNING_RATE}\n")
    
    gru_model, gru_history = train_keras_standard(
        model=gru_model,
        train_data=train_loader,
        val_data=val_loader,
        num_epochs=NUM_EPOCHS,
        model_save_path=GRU_MODEL_PATH,
        history_save_path=GRU_HISTORY_PATH,
        patience=PATIENCE
    )
    print("\n✅ Huấn luyện GRU hoàn thành.\n")

    # --- BƯỚC 5: ĐÁNH GIÁ MODEL TRÊN TEST SET ---
    print(">>> BƯỚC 5: ĐÁNH GIÁ MODEL GRU TRÊN TẬP TEST...")
    gru_results = evaluate_tf(
        model=gru_model,
        test_data=test_loader,
        n_labels=N_LABELS,
        label_names=LABEL_NAMES
    )
    
    # --- KẾT QUẢ FINAL ---
    print("\n" + "="*70)
    print("🏆 KẾT QUẢ BÀI 2 - MẠNG GRU (5 LAYERS)")
    print("="*70)
    print(f"✅ Test Accuracy: {gru_results['test_accuracy']*100:.2f}%")
    print(f"✅ Test F1-Score (Macro): {gru_results['test_f1_macro']:.4f}")
    print(f"✅ Model được lưu tại: {GRU_MODEL_PATH}")
    print(f"✅ History được lưu tại: {GRU_HISTORY_PATH}")
    print("="*70 + "\n")

except Exception as e:
    print(f"\n❌ [LỖI BÀI 2]: {e}")
    import traceback
    traceback.print_exc()

 🟢 BÀI 2: HUẤN LUYỆN MẠNG GRU (5 LAYERS)

>>> BƯỚC 1: ĐANG TẢI VÀ XỬ LÝ DỮ LIỆU...
✅ Đã load tập 'Train': 11426 dòng.
✅ Đã load tập 'Dev': 1583 dòng.
✅ Đã load tập 'Test': 3166 dòng.
✅ Vocab khởi tạo từ 3 DataFrame
  ℹ️ Tự động set max_seq_len = 160
  ℹ️ Tự động set max_seq_len = 170
  ℹ️ Tự động set max_seq_len = 100
 - Vocab Size: 2879
 - Labels: 4
>>> Dữ liệu đã sẵn sàng.

>>> BƯỚC 2: KHỞI TẠO MODEL GRU...
✅ Model GRU đã khởi tạo.

[Kiến trúc Model GRU]:


Model: "gru_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (1, 10, 300)           │       863,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (1, 10, 256)           │       428,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (1, 10, 256)           │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (1, 10, 256)           │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (1, 10, 256)           │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (1, 256)               │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (1, 4)                 │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,872,280 (10.96 MB)

 Trainable params: 2,872,280 (10.96 MB)

 Non-trainable params: 0 (0.00 B)


>>> BƯỚC 3: COMPILE MODEL GRU...
✅ Model GRU đã compile.

>>> BƯỚC 4: HUẤN LUYỆN MODEL GRU...
Hyperparameters: Epochs=20, Batch Size=64, Patience=10
Learning Rate: 0.001

--- Bắt đầu training (Sử dụng Keras Standard) ---
Lưu model tốt nhất tại: best_gru_model.keras
Lưu lịch sử training tại: gru_history.json
Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - f1_score: 0.7568 - loss: 0.6776
Epoch 1: val_loss improved from None to 0.39471, saving model to best_gru_model.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 1174s 7s/step - f1_score: 0.8085 - loss: 0.5333 - val_f1_score: 0.8528 - val_loss: 0.3947
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - f1_score: 0.8726 - loss: 0.3493
Epoch 2: val_loss improved from 0.39471 to 0.37241, saving model to best_gru_model.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 1025s 6s/step - f1_score: 0.8741 - loss: 0.3563 - val_f1_score: 0.8591 - val_loss: 0.3724
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - f1_score: 0.8924 - loss: 0.2970
Epoch 3: val_loss did 